## Mortality Training
1. Load the MIMIC III Dataset
2. Normalize Data
3. Load into Pytorch Dataloader
4. Train
5. Evaluate

In [12]:
from comet_ml import Experiment
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

from core.model import SAnD
from mimic3_benchmarks.mimic3benchmark.readers import PhenotypingReader as Reader
from mimic3_benchmarks.mimic3models.preprocessing import Discretizer, Normalizer
from utils.functions import get_weighted_sampler, get_weights
from utils.trainer import NeuralNetworkClassifier
from utils.batch_gen import BatchGen
from utils.phenotyping_utils import load_data


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
task = "phenotyping"

## Load Data
Using the [MimicIII Benchmark's](https://github.com/YerevaNN/mimic3-benchmarks) InHospitalMortalityReader and associated functions, load the data into memory.

If you are following along you will need to arrange the data following the directions on the link.


In [2]:
train_reader = Reader(dataset_dir=f"mimic3_benchmarks/data/{task}/train",
                              listfile=f"mimic3_benchmarks/data/{task}/train_listfile.csv")
val_reader = Reader(dataset_dir=f"mimic3_benchmarks/data/{task}/train",
                              listfile=f"mimic3_benchmarks/data/{task}/val_listfile.csv")
test_reader = Reader(dataset_dir=f"mimic3_benchmarks/data/{task}/test",
                              listfile=f"mimic3_benchmarks/data/{task}/test_listfile.csv")

### Example data

In [3]:
ex = pd.DataFrame.from_dict(train_reader.read_example(0)['X'])
ex.columns = train_reader.read_example(0)["header"]
print(train_reader.read_example(0)['y'])
ex

[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


,Hours,Capillary refill rate,Diastolic blood pressure,Fraction inspired oxygen,Glascow coma scale eye opening,Glascow coma scale motor response,Glascow coma scale total,Glascow coma scale verbal response,Glucose,Heart Rate,Height,Mean blood pressure,Oxygen saturation,Respiratory rate,Systolic blood pressure,Temperature,Weight,pH
0,0.30527777777777776,,86.0,,1 No Response,5 Localizes Pain,7,1.0 ET/Trach,,92,,104,100.0,14,141.0,,,
1,0.47194444444444444,,,,,,,,142.0,,,,100.0,14,,,,
2,0.6719444444444445,,,,,,,,,,,,,,,,,7.43
3,1.3052777777777778,,90.0,,,,,,,98,,109,100.0,14,144.0,38.16669845581055,,
4,2.3052777777777775,,90.0,,,,,,,99,,109,100.0,14,147.0,,,
5,3.3052777777777775,,93.0,,,,,,,106,,113,100.0,14,152.0,39.11111195882162,,
6,4.3052777777777775,,88.0,,,,,,,106,,106,100.0,14,145.0,,,
7,5.3052777777777775,,86.0,,3 To speech,5 Localizes Pain,9,1.0 ET/Trach,,103,,103,100.0,14,141.0,39.11111195882162,,
8,5.8052777777777775,,85.0,,,,,,,112,,102,100.0,16,135.0,,,
9,6.3052777777777775,,88.0,,,,,,,105,,110,100.0,9,151.0,,,


### Normalize and Discretize data
From the MIMIC-III Benchmark repo

In [4]:
discretizer = Discretizer(timestep=1.0,
                          store_masks=True,
                          impute_strategy='previous',
                          start_time='zero')

discretizer_header = discretizer.transform(train_reader.read_example(0)["X"])[1].split(',')
cont_channels = [i for (i, x) in enumerate(discretizer_header) if x.find("->") == -1]

In [5]:
normalizer = Normalizer(fields=cont_channels)  # choose here which columns to standardize
normalizer_state = 'train/ph_ts1.0.input_str-previous.start_time-zero.normalizer'
normalizer.load_params(normalizer_state)

In [15]:
batch_size = 128
max_seq_len = 256
small_part = True

train_raw = load_data(train_reader, discretizer, normalizer, max_seq_len, small_part)
val_raw = load_data(val_reader, discretizer, normalizer, max_seq_len, small_part)
test_raw = load_data(test_reader, discretizer, normalizer, max_seq_len, small_part)

train_ds = TensorDataset(train_raw[0], train_raw[1])
val_ds = TensorDataset(val_raw[0], val_raw[1])
test_ds = TensorDataset(test_raw[0], test_raw[1])

train_loader = DataLoader(train_ds, batch_size=batch_size)#sampler=get_weighted_sampler(y_train))
val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_ds, batch_size=batch_size)

N, seq_len, feature_count = train_raw[0].shape
print(N, seq_len, feature_count)

ValueError: not enough values to unpack (expected 3, got 2)

## Create model, Train, and Evaluate
Train the model, this will export results to Comet ML.
Creating is done in the same step such that a new experiment made each time
Evaluation is done in the same step such that the Comet ML experiment is also deleted.

In [13]:
n_heads = 8
factor = 120 # M
num_class = 25
num_layers = 2 # N
epochs = 30
betas = (0.9, 0.98)
lr = 0.0005
eps = 4e-09
weight_decay = 5e-4
dropout = 0.4

experiment = Experiment(
    api_key="eQ3INeSsFGUYKahSdEtjhry42",
    project_name="general",
    workspace="samdoud"
)

clf = NeuralNetworkClassifier(
    SAnD(feature_count, seq_len, n_heads, factor, num_class, num_layers, dropout_rate=dropout),
    nn.BCEWithLogitsLoss(),
    optim.Adam, optimizer_config={
        "lr": lr, "betas": betas, "eps": eps, "weight_decay": weight_decay},
    experiment=experiment
)

clf.fit(
    {
        "train": train_loader,
        "val": val_loader
    },
    validation=True,
    epochs=epochs,
    verbose=True,
)

clf.evaluate(test_loader)
experiment.end()

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/samdoud/general/c6305004fc1649369bf5560d61f3697d

Training - Epochs: 001/030: 100%|█████████▉| 29184/29250 [08:59<00:00, 2363.28it/s]

Train finished. Accuracy: 0.5455 MSE: 0.0000 AUROC: 0.0000 AUPRC: 0.0000


Training - Epochs: 001/030: 100%|██████████| 29250/29250 [09:00<00:00, 54.07it/s]  


Validation finished. Accuracy: 0.6646 MSE: 0.0000 AUROC: 0.0000 AUPRC: 0.0000


Training - Epochs: 002/030: 100%|█████████▉| 29184/29250 [00:11<00:00, 2537.39it/s]

...

Training - Epochs: 030/030: 100%|██████████| 29250/29250 [00:12<00:00, 2274.27it/s]


Validation finished. Accuracy: 0.7255 MSE: 0.0000 AUROC: 0.0000 AUPRC: 0.0000


Evaluating: 100%|██████████| 6281/6281 [00:01<00:00, 4611.65it/s]
COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/samdoud/general/c6305004fc1649369bf5560d61f3697d
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     test_loss [50]         : (1.1974750757217407, 62.925976037979126)
COMET INFO:     train_accuracy [30]    : (0.5454524357840066, 0.6757104408058814)
COMET INFO:     train_loss [7557]      : (0.3341692388057709, 280.1351013183594)
COMET INFO:     validate_accuracy [30] : (0.6645940886693542, 0.7257305206106658)
COMET INFO:     validate_loss [1500]   : (0.8510227799415588, 12.934517860412598)
COMET INFO:   Parameters:
COMET INFO:     batch_size    : 128
COMET INFO:     betas         : (0.9, 0.98)
COMET INFO:     epochs        : 30
COMET INFO:     eps           : 4e-09
COMET INFO:

error on test
Evaluation finished. Accuracy: 0.7255 MSE: 0.0000 AUROC: 0.0000 AUPRC: 0.0000


COMET INFO: Uploading 785 metrics, params and output messages


ModuleNotFoundError: No module named 'mimic3models'